# Working with Events

Methods for retrieving open access data.

A database schema diagram for production instances of CDP may be found [here](https://github.com/CouncilDataProject/cdptools/blob/master/docs/resources/database_diagram.pdf).

# Connecting to resources

Having access to both the CDP instance's database and file store with make accessing and using the transcripts easiest. It is recommended to read the database, file store, and transcript usage notebooks prior to working through this one.

For details on database usage, refer to the notebook example on database basics [here](./database.ipynb).

For details on file store usage, refer to the notebook example on file store basics [here](./file_store.ipynb).

**Note:** This notebook connects to the staging instance of Seattle's Firestore database and file store. To use production data, connect to the Cloud Firestore instance: `cdp-seattle`. To use production files, connect to the GCS instance: `cdp-seattle.appspot.com`.

In [1]:
from cdptools.databases.cloud_firestore_database import CloudFirestoreDatabase
from cdptools.file_stores.gcs_file_store import GCSFileStore
import pandas as pd

db = CloudFirestoreDatabase("stg-cdp-seattle")
fs = GCSFileStore("stg-cdp-seattle.appspot.com")
db, fs

(<CloudFirestoreDatabase [stg-cdp-seattle]>,
 <GCSFileStore [stg-cdp-seattle.appspot.com]>)

### Get all votes for a single person

Query and merge the `person` and `vote` tables.

In [2]:
# Filtering not supported for open access API yet, get all people then dataframe filter to target
people = pd.DataFrame(db.select_rows_as_list("person"))
teresa = people.loc[people["full_name"] == "Teresa Mosqueda"]

# Pull voting data
votes = pd.DataFrame(db.select_rows_as_list("vote"))

# Merge data
teresa_voting_record = teresa.merge(
    votes,
    left_on="person_id",
    right_on="person_id",
    suffixes=("_person", "_vote")
)

teresa_voting_record.head()

created_person                        email        full_name  \
0 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
1 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
2 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
3 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
4 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   

   legistar_person_id                             person_id         phone  \
0                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
1                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
2                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
3                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
4                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   

                                   website               created_vote  \
0  http://www.seattle.gov/council/mosqueda 2019-07-20 23:34:14.510429   
1  http://www.seattle.gov/council/mosqueda 2019-07-21 00:29:28.257667   
2  http://www.seattle.gov/council/mosqueda 2019-07-20 23:22:20.345379   
3  http://www.seattle.gov/council/mosqueda 2019-07-20 23:28:34.894883   
4  http://www.seattle.gov/council/mosqueda 2019-07-21 00:13:15.404858   

   decision                 event_minutes_item_id  \
0  In Favor  650b4bc0-6dfe-4c8e-b987-326e6b326d6f   
1  In Favor  9679b518-db5b-4c3e-ab54-e35a933e1ccc   
2  In Favor  01c4b5c4-2d02-44b2-808b-e8f71c77c70d   
3  In Favor  cfe4d044-beab-4b05-9e74-27eefdcaeda2   
4  In Favor  8a5309d2-2b18-4ce2-bbb7-e66626ae9565   

   legistar_event_item_vote_id                               vote_id  
0                        46473  02ea892d-c24d-4d91-91df-a29cd6735080  
1                        46659  0b001131-b59c-413f-8d8e-07b6b5df20f6  
2                        46725  105291a1-ce97-452c-b0ee-91ce1a314471  
3                        47003  1564337b-f145-4f61-9d14-875faa8351c6  
4                        46978  18bdd900-23f2-42a8-b30b-5ee7fd18abff

This is great, but it's missing context of what was actually being voted on. Currently we can see that she voted on instances of `event_minutes_item`, but what were those. Follow the same procedure of `select_rows_as_list` -> `merge` to join the data of the other tables to Teresa's voting record.

In [3]:
# Get event minutes item context data
event_minutes_items = pd.DataFrame(db.select_rows_as_list("event_minutes_item"))
minutes_items = pd.DataFrame(db.select_rows_as_list("minutes_item"))
events = pd.DataFrame(db.select_rows_as_list("event"))

# Merge minutes item data
event_minutes_items = event_minutes_items.merge(
    minutes_items,
    left_on="minutes_item_id",
    right_on="minutes_item_id",
    suffixes=("_event_minutes_item", "_minutes_item")
)

# Merge event data
event_minutes_items = event_minutes_items.merge(
    events,
    left_on="event_id",
    right_on="event_id",
    suffixes=("_event_minutes_item", "_event")
)

# Merge above event minutes item data with Teresa's voting record
teresa_voting_record = teresa_voting_record.merge(
    event_minutes_items,
    left_on="event_minutes_item_id",
    right_on="event_minutes_item_id",
    suffixes=("_voting_record", "_event_minutes_item")
)

# Sort by event datetime
teresa_voting_record = teresa_voting_record.sort_values(by=["event_datetime"])
teresa_voting_record.head()

created_person                        email        full_name  \
0  2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
21 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
20 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
17 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   
13 2019-07-20 23:22:09.665851  Teresa.Mosqueda@seattle.gov  Teresa Mosqueda   

    legistar_person_id                             person_id         phone  \
0                  594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
21                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
20                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
17                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   
13                 594  c4ded0b8-912a-4f68-b436-0d52d5ca66e0  206-684-8806   

                                    website               created_vote  \
0   http://www.seattle.gov/council/mosqueda 2019-07-20 23:34:14.510429   
21  http://www.seattle.gov/council/mosqueda 2019-07-20 23:34:00.759789   
20  http://www.seattle.gov/council/mosqueda 2019-07-20 23:33:57.044317   
17  http://www.seattle.gov/council/mosqueda 2019-07-20 23:34:11.733353   
13  http://www.seattle.gov/council/mosqueda 2019-07-20 23:34:08.593559   

   decision_voting_record                 event_minutes_item_id  ...  \
0                In Favor  650b4bc0-6dfe-4c8e-b987-326e6b326d6f  ...   
21               In Favor  ca2038ca-7bc0-43f1-850c-489cf69b9990  ...   
20               In Favor  544e8efd-c167-4ae6-8a83-b2dc5fe61694  ...   
17               In Favor  563774a9-f15d-4592-b034-35838846bd59  ...   
13               In Favor  99d5e97c-00cb-4191-be4a-fcd59b585a6e  ...   

                                                 name  \
0   Reappointment of Torgie Madison as member, Com...   
21                                          CB 119546   
20                                          CB 119539   
17  Appointment of Rene J. Peters, Jr. as member, ...   
13                                          CB 119531   

                                      agenda_file_uri  \
0   http://legistar2.granicus.com/seattle/meetings...   
21  http://legistar2.granicus.com/seattle/meetings...   
20  http://legistar2.granicus.com/seattle/meetings...   
17  http://legistar2.granicus.com/seattle/meetings...   
13  http://legistar2.granicus.com/seattle/meetings...   

                                 body_id                    created  \
0   f934cfa0-4285-404f-8072-4163df0a03b3 2019-07-20 23:33:41.354756   
21  f934cfa0-4285-404f-8072-4163df0a03b3 2019-07-20 23:33:41.354756   
20  f934cfa0-4285-404f-8072-4163df0a03b3 2019-07-20 23:33:41.354756   
17  f934cfa0-4285-404f-8072-4163df0a03b3 2019-07-20 23:33:41.354756   
13  f934cfa0-4285-404f-8072-4163df0a03b3 2019-07-20 23:33:41.354756   

        event_datetime  legistar_event_id  \
0  2019-06-24 14:00:00               4007   
21 2019-06-24 14:00:00               4007   
20 2019-06-24 14:00:00               4007   
17 2019-06-24 14:00:00               4007   
13 2019-06-24 14:00:00               4007   

                                  legistar_event_link  \
0   https://seattle.legistar.com/MeetingDetail.asp...   
21  https://seattle.legistar.com/MeetingDetail.asp...   
20  https://seattle.legistar.com/MeetingDetail.asp...   
17  https://seattle.legistar.com/MeetingDetail.asp...   
13  https://seattle.legistar.com/MeetingDetail.asp...   

                                     minutes_file_uri  \
0   http://legistar2.granicus.com/seattle/meetings...   
21  http://legistar2.granicus.com/seattle/meetings...   
20  http://legistar2.granicus.com/seattle/meetings...   
17  http://legistar2.granicus.com/seattle/meetings...   
13  http://legistar2.granicus.com/seattle/meetings...   

                                           source_uri  \
0   http://www.seattlechannel.org/FullCouncil?vide...   
21  http:

In [4]:
teresa_voting_record.loc[0]

created_person                                        2019-07-20 23:22:09.665851
email                                                Teresa.Mosqueda@seattle.gov
full_name                                                        Teresa Mosqueda
legistar_person_id                                                           594
person_id                                   c4ded0b8-912a-4f68-b436-0d52d5ca66e0
phone                                                               206-684-8806
website                                  http://www.seattle.gov/council/mosqueda
created_vote                                          2019-07-20 23:34:14.510429
decision_voting_record                                                  In Favor
event_minutes_item_id                       650b4bc0-6dfe-4c8e-b987-326e6b326d6f
legistar_event_item_vote_id                                                46473
vote_id                                     02ea892d-c24d-4d91-91df-a29cd6735080
created_event_minutes_item  